<a href="https://colab.research.google.com/github/jhuarancca/Data-Science-for-Healthcare-Claims-data/blob/main/Logistic%2Bregression%2Bwith%2Bpython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import the relevant libraries**

In [1]:
import pandas as pd
import numpy as np
import io # If you are not using Google Colab, skip the last sentence containing 'import io'
import statsmodels.api as sm # this library has now been added because it is needed for logistic regression 


**Upload the dataset**

In [5]:
from google.colab import files
raw_claims_data = files.upload()

# If you are not using Google Colab, skip this code cell


Saving OBGYN_practice_dataset_with_categories (2).csv to OBGYN_practice_dataset_with_categories (2).csv


**Import the dataset**

In [11]:
raw_data = pd.read_csv('OBGYN_practice_dataset_with_categories.csv')


# If you are not using Google Colab, use a different code:

# raw_data = pd.read_csv(r'C:\Users\location of the csv file on your computer\\OBGYN_with categories.csv')


**Inspect the dataset**

In [12]:
raw_data

,person_id,procedure_date,procedure_concept_id,concept_name,amount,total_paid,provider_id,provider_name,category according to domain expert
0,11248,8/3/2020,2004407,Amputation of cervix,1,839,45192,Baptist Health & Medical Center,Procedure
1,11874,4/6/2020,2004407,Amputation of cervix,1,839,52356,Boudica Female Care,Procedure
2,12745,12/20/2019,2004407,Amputation of cervix,1,839,24023,dr. Mariano & Co.,Procedure
3,13133,7/29/2019,2004407,Amputation of cervix,1,839,24023,dr. Mariano & Co.,Procedure
4,13844,8/31/2020,2004407,Amputation of cervix,1,839,45192,Baptist Health & Medical Center,Procedure
...,...,...,...,...,...,...,...,...,...
52188,19298,9/26/2019,45890386,Abortion Procedures,1,1129,45192,Baptist Health & Medical Center,Procedure
52189,19406,1/18/2020,45890386,Abortion Procedures,1,1129,45192,Baptist Health & Medical Center,Procedure
52190,19435,8/26/2020,45890386,Abortion Procedures,1,1129,4823,East Westfield Health Services,Procedure
52191,19656,4/30/2020,45890386,Abortion Procedures,1,1129,45192,Baptist Health & Medical Center,Procedure


**Create a pivot table**

In [13]:
pivot = pd.pivot_table(raw_data, values = 'amount', index=['person_id'], columns = 'category according to domain expert',aggfunc=np.sum).reset_index()

**Inspect the pivot table**

In [14]:
pivot

category according to domain expert,person_id,Consultation,Delivery,Diagnostic,Procedure
0,10429,4.0,NaN,3.0,NaN
1,10430,12.0,1.0,6.0,NaN
2,10431,10.0,1.0,6.0,NaN
3,10432,6.0,NaN,6.0,NaN
4,10433,1.0,NaN,NaN,NaN
...,...,...,...,...,...
9258,19688,2.0,1.0,8.0,NaN
9259,19689,2.0,NaN,2.0,NaN
9260,19690,12.0,1.0,6.0,NaN
9261,19691,1.0,NaN,NaN,NaN


**Replace blank cells with zero**

In [15]:
raw_data = pivot.fillna(0)

**Inspect the new dataset**

In [16]:
raw_data

category according to domain expert,person_id,Consultation,Delivery,Diagnostic,Procedure
0,10429,4.0,0.0,3.0,0.0
1,10430,12.0,1.0,6.0,0.0
2,10431,10.0,1.0,6.0,0.0
3,10432,6.0,0.0,6.0,0.0
4,10433,1.0,0.0,0.0,0.0
...,...,...,...,...,...
9258,19688,2.0,1.0,8.0,0.0
9259,19689,2.0,0.0,2.0,0.0
9260,19690,12.0,1.0,6.0,0.0
9261,19691,1.0,0.0,0.0,0.0


**Retrieve the names of the columns**

In [17]:
raw_data.columns

Index(['person_id', 'Consultation', 'Delivery', 'Diagnostic', 'Procedure'], dtype='object', name='category according to domain expert')

**Rearrange the columns to place the target variable at the end**

In [18]:
dataset = raw_data[['person_id', 'Consultation',  'Diagnostic', 'Procedure','Delivery']]

**Inspect the new dataset**

In [19]:
dataset

category according to domain expert,person_id,Consultation,Diagnostic,Procedure,Delivery
0,10429,4.0,3.0,0.0,0.0
1,10430,12.0,6.0,0.0,1.0
2,10431,10.0,6.0,0.0,1.0
3,10432,6.0,6.0,0.0,0.0
4,10433,1.0,0.0,0.0,0.0
...,...,...,...,...,...
9258,19688,2.0,8.0,0.0,1.0
9259,19689,2.0,2.0,0.0,0.0
9260,19690,12.0,6.0,0.0,1.0
9261,19691,1.0,0.0,0.0,0.0


**Inspect if the target is binary**

In [20]:
dataset.Delivery.value_counts()

 

0.0    8110
1.0    1143
2.0      10
Name: Delivery, dtype: int64

**Convert the target to binary values**

In [21]:
Data_with_binary_targets = dataset.copy()
Binary = Data_with_binary_targets['Delivery_binary'] = np.where(Data_with_binary_targets['Delivery'] > 0, 1, 0)
Data_with_binary_targets['Delivery_binary'] = Binary
Data_with_binary_targets


category according to domain expert,person_id,Consultation,Diagnostic,Procedure,Delivery,Delivery_binary
0,10429,4.0,3.0,0.0,0.0,0
1,10430,12.0,6.0,0.0,1.0,1
2,10431,10.0,6.0,0.0,1.0,1
3,10432,6.0,6.0,0.0,0.0,0
4,10433,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...
9258,19688,2.0,8.0,0.0,1.0,1
9259,19689,2.0,2.0,0.0,0.0,0
9260,19690,12.0,6.0,0.0,1.0,1
9261,19691,1.0,0.0,0.0,0.0,0


**Confirm the binary values by counting the values in the target column**

In [22]:
Data_with_binary_targets.Delivery_binary.value_counts()

0    8110
1    1153
Name: Delivery_binary, dtype: int64

**Remove the column 'Delivery' that is non-binary**

In [23]:
Dataframe = Data_with_binary_targets.drop(columns=['Delivery'])

**Assign the predictive variables (x1) and the target variable (y)**

In [24]:
x1 = Dataframe.iloc[:,1:-1]
y = Dataframe.iloc[:,-1]


**Confirm x1**

In [25]:
x1

category according to domain expert,Consultation,Diagnostic,Procedure
0,4.0,3.0,0.0
1,12.0,6.0,0.0
2,10.0,6.0,0.0
3,6.0,6.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9258,2.0,8.0,0.0
9259,2.0,2.0,0.0
9260,12.0,6.0,0.0
9261,1.0,0.0,0.0


**Confirm y**

In [26]:
y

0       0
1       1
2       1
3       0
4       0
       ..
9258    1
9259    0
9260    1
9261    0
9262    0
Name: Delivery_binary, Length: 9263, dtype: int64

**Add a constant** (For more info, see https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html )

In [27]:
x = sm.add_constant(x1)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


**Apply logistic regression to the data**

In [28]:
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()


Optimization terminated successfully.
         Current function value: 0.216019
         Iterations 8


**Inspect the summary of the logistic regression**

In [29]:
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        Delivery_binary   No. Observations:                 9263
Model:                          Logit   Df Residuals:                     9259
Method:                           MLE   Df Model:                            3
Date:                Tue, 14 Feb 2023   Pseudo R-squ.:                  0.4251
Time:                        21:55:52   Log-Likelihood:                -2001.0
converged:                       True   LL-Null:                       -3480.5
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -4.1147      0.080    -51.640      0.000      -4.271      -3.959
Consultation     0.1225      0.015      8.441      0.000       0.094       0.151
Diagnostic       0.6124      0.024     26.007      0.000       0.566       0.659
Procedure       -1.1767      0.138     -8.549      0.000      -1.446      -0.907
================================================================================
"""

**Evaluate the results with a confusion matrix**



In [30]:
cm = results_log.pred_table()
cm_df = pd.DataFrame(cm)
cm_df.columns = ['Predicted 0', 'Predicted 1']
cm_df = cm_df.rename(index={0:'Actual 0', 1:'Actual 1'})
cm_df  


,Predicted 0,Predicted 1
Actual 0,7932.0,178.0
Actual 1,540.0,613.0


**Calculate the accuracy of the model**

In [31]:
(cm[0,0]+cm[1,1])/np.sum([cm]) 

0.9224873151246896